# Hello Genetic Programming

### Definitions 

- **Genetic Algorithms** (GA) are search algorithms that mimic the process of natural evolution (blah blah blah) where each individual is a candidate solution: individuals are generally raw data in whatever encoding format has been defined.

- **Genetic Programming (GP)** is considered a special case of GA, where each individual is a computer program (not just raw data). GP explore the algorithmic search space and evolve computer programs to perform a defined task.

<hr/>
<div class="container-fluid">
  <div class='well'>
      <div class="row">
          <div class="col-md-12" align='center'>
              <img align='center'alt=“Genetic Programming“ style="border-width:0" src="https://cdn-images-1.medium.com/max/1600/1*BYDJpa6M2rzWNSurvspf8Q.png"/>
          </div>
      </div>
  </div>
</div>

### Terms

 - **Mutation**: a mutation is a genetic operator used to maintain genetic diversity from one generation of a population of genetic algorithm chromosomes to the next. It is analogous to biological mutation. Mutation alters one or more gene values in a chromosome from its initial state. In mutation, the solution may change entirely from the previous solution. Hence GA can come to a better solution by using mutation. 
 
 - **Crossover**: is a genetic operator used to vary the programming of a chromosome or chromosomes from one generation to the next. It is analogous to reproduction and biological crossover, upon which genetic algorithms are based. Crossover is a process of taking more than one parent solution and producing a child solution from them. 
 
 - **Fitness**: the "fitness" is a function which takes a candidate solution to the problem as input and produces as output how “fit” our how “good” the solution is with respect to the problem in consideration.

### Pros

- Faster (and lower memory requirements) than searching a very large search space.
- Easy, in that if your candidate representation and fitness function are correct, a solution can be found without any explicit analytical work.

### Caveats

- GAs do not scale well with complexity (i.e. slow convergence towards solutions). 
- GAs may converge towards a local optima / minima, or even arbitrary points rather than the global optimum of the problem. Could be solved by using a different fitness function to explore the solution landscape, increasing the mutation rate or modifying the selection criterion. However, there is no known general solution to this problem (i.e. the “No Free Lunch Theorem”).
- GAs are difficult to work with dynamic data sets.
- GAs cannot effectively solve decision problems.
- Depending on the problem, other optimization algorithms may be better suited to evaluating and converging towards a solution than GAs.

### Let's try to implement this in Python

First, let's setup our imports:

In [1]:
import random
import string
from pprint import pprint
from datetime import datetime


POPULATION = 20


# these are the characters which make up the "DNA" of our example
GENES = string.letters + '!, '

Just like in biological evolution, we're applying a fitness criterion - which in this case is lexicographical distance:

In [2]:
def calc_fitness(source, target):
    # Compare the lexicographic distance between a source and the target
    # and summate this difference squared.
    fitval = 0
    for i in xrange(len(source)):
        fitval += (ord(target[i]) - ord(source[i])) ** 2
    return fitval

Similarly, we also need to mutate each individual and calculate their respective fitness:

In [4]:
def mutate(parent1, parent2, target):
    child_dna = parent1['dna'][:]  # `child_dna` is a copy of `parent1` at this point

    # now attempt cross-over
    start = random.randint(0, len(parent2['dna']) - 1)
    stop = random.randint(0, len(parent2['dna']) - 1)

    # since we're using the indices to copy over parts of each parent we
    # need the start to be less than stop. This condition ensures this by
    # switching the positions.
    if start > stop:
        stop, start = start, stop
    
    # now copy over the relevant parts
    child_dna[start:stop] = parent2['dna'][start:stop]

    # now mutate one position
    char_pos = random.randint(0, len(child_dna) - 1)
    #child_dna[char_pos] = GENES[random.randint(0, len(GENES) -1)]
    child_dna[char_pos] = chr(ord(child_dna[char_pos]) + random.randint(-1, 1))
    child_fitness = calc_fitness(child_dna, target)
    return {'dna': child_dna, 'fitness': child_fitness}

Then, we need to determine how we select individuals from the pool for mating:

In [5]:
def random_parent(gene_pool):
    # randomly chose a parent from the pool
    index = random.randint(0, len(gene_pool)-1)
    return gene_pool[index]


# This is a convenience function to be used for displaying the details of a given 
# genetic pool at some generation.
def print_gene_pool(gene_pool, generation):
    for candidate in gene_pool:
        print '%i %i %s' % (generation, 
                            candidate['fitness'], 
                            ''.join(candidate['dna']))
    print


Now we evolve the pool, mutating the individual members and selecting based on our fitness criterion:

In [7]:
random.seed(314159265)  # for reproducibility
target = 'Hello, world!'

# create a starting pool of dna
gene_pool = list()
for i in xrange(0, POPULATION):
    dna = [random.choice(GENES) for g in xrange(0, len(target))]
    fitness = calc_fitness(dna, target)
    candidate = {'dna': dna, 'fitness': fitness}
    gene_pool.append(candidate)

    
# Evolve the pool until one the candidates reaches the 
generation = 1
silent = False
fittest = list()
while True:
    # sort the gene pool by fitness in ascending order. This means that the
    # 1st element (index 0) is always the fittest individual
    gene_pool.sort(key=lambda candidate: candidate['fitness'])
    fittest.append(gene_pool[0])
    if not silent:
        print_gene_pool(gene_pool, generation)

    parent1 = random_parent(gene_pool)
    parent2 = random_parent(gene_pool)

    child = mutate(parent1, parent2, target)

    # compare the fitness of the child relative to that of the pool
    # and if the child's fitness is less than most unfit individual in the
    # pool, it's added to the pool - i.e. genetic elitism.
    if child['fitness'] < gene_pool[-1]['fitness']:
        gene_pool[-1] = child

    # terminate when we've reached the goal (fitness = 0)
    if gene_pool[0]['fitness'] == 0:
        break

    generation += 1
    
# display the fittest individuals
for generation, best in enumerate(fittest):
    if not silent:
        print generation+1, ''.join(best['dna']), best['fitness']

1 4295 Dofsi,!mMlRlN
1 9940 GouVIDGCRcYmH
1 10040 PtyhIFNdORHiI
1 10785 pbhfvJLVMvPWX
1 12992 RBIZnTgjecaQS
1 13727 UXefjdGAxHCDC
1 14363 jkiVbLasaRLZg
1 14550 vioLXpPFxbbeB
1 15925 hdhqD,veOmpdb
1 15939 synPRuMgjdIVV
1 16107 hxjaCGtIZsaJH
1 16135 HEPtQOFOMxZax
1 18046 sEIZPBXoEisws
1 20084 AUWpvnfZUCw!J
1 21898 BA!cdVzkcFeeT
1 22874 qjWetXUAV,Iep
1 25718 hVdMdvoaIeRCy
1 28127  DRR cVZFPpuz
1 28565 tolztQxt dI g
1 30485 oMP!XhY!wvXes

2 4295 Dofsi,!mMlRlN
2 9940 GouVIDGCRcYmH
2 10040 PtyhIFNdORHiI
2 10785 pbhfvJLVMvPWX
2 12992 RBIZnTgjecaQS
2 13727 UXefjdGAxHCDC
2 14363 jkiVbLasaRLZg
2 14550 vioLXpPFxbbeB
2 15925 hdhqD,veOmpdb
2 15939 synPRuMgjdIVV
2 16107 hxjaCGtIZsaJH
2 16135 HEPtQOFOMxZax
2 18046 sEIZPBXoEisws
2 20084 AUWpvnfZUCw!J
2 21498 RBRR cVZFPbQS
2 21898 BA!cdVzkcFeeT
2 22874 qjWetXUAV,Iep
2 25718 hVdMdvoaIeRCy
2 28127  DRR cVZFPpuz
2 28565 tolztQxt dI g

3 4295 Dofsi,!mMlRlN
3 9940 GouVIDGCRcYmH
3 10040 PtyhIFNdORHiI
3 10785 pbhfvJLVMvPWX
3 12992 RBIZnTgjecaQS
3 13727 UXefjd

71 4295 Dofsi,!mMlRlN
71 4295 Dofsi,!mMlRlN
71 4295 Dofsi,!mMlRlN

72 2652 Dofsi,!mMuafA
72 2720 Tofsi,!mNvafA
72 2812 Uofsi,!mMvafA
72 3294 Dofsi,!mMlRlA
72 3315 Dofsi,!lMlRlA
72 3426 Dofsi,!mMlRlC
72 3426 Dofsi,!mMlRlC
72 3442 TofsiF!mMvbfA
72 3443 Tofsi,!lMlRlA
72 3447 Dofsi,!lMlRlC
72 3554 DofsiG!lNlalC
72 3656 Dofsi,"mNvalN
72 3662 Dofsi,!mNwalN
72 3662 Dofsi,!mNwalN
72 4056 DofsiF!lNlRlC
72 4228 Dofsi,!mNlRlN
72 4244 Dofsi,!mMlSlN
72 4295 Dofsi,!mMlRlN
72 4295 Dofsi,!mMlRlN
72 4295 Dofsi,!mMlRlN

73 2652 Dofsi,!mMuafA
73 2720 Tofsi,!mNvafA
73 2812 Uofsi,!mMvafA
73 3294 Dofsi,!mMlRlA
73 3315 Dofsi,!lMlRlA
73 3426 Dofsi,!mMlRlC
73 3426 Dofsi,!mMlRlC
73 3442 TofsiF!mMvbfA
73 3443 Tofsi,!lMlRlA
73 3447 Dofsi,!lMlRlC
73 3554 DofsiG!lNlalC
73 3656 Dofsi,"mNvalN
73 3662 Dofsi,!mNwalN
73 3662 Dofsi,!mNwalN
73 4056 DofsiF!lNlRlC
73 4228 Dofsi,!mNlRlN
73 4228 Dofsi,!mNlRlN
73 4244 Dofsi,!mMlSlN
73 4295 Dofsi,!mMlRlN
73 4295 Dofsi,!mMlRlN

74 2652 Dofsi,!mMuafA
74 2720 Tofsi,!mNvafA
74 2812

138 2602 Dogsi,!mMlae@
138 2616 Dofsi,!mMlaf@
138 2617 Dofsi+!mMlaf@
138 2632 Eofri,!mMuafA
138 2639 Dofri,!mMuafA
138 2641 Dogsi,!mMuafA
138 2641 Dogsi,!mMuafA
138 2649 Cofth,!mNlafA
138 2652 Dofsi,!mMuafA
138 2652 Dofsi,!mMuafA
138 2652 Dofsi,!mMuafA
138 2655 Dofsi,"mMuafA
138 2655 Dofsi,"mMuafA
138 2665 Dofsh,!mMuafA
138 2665 Dofsh,!mMuafA
138 2667 Dofti,!mNvalA
138 2667 Dofti,!mMuafA
138 2674 Dofti,!mMvafA
138 2676 Cofti,!mMuafA

139 2588 Cofti,!mNubfA
139 2602 Dogsi,!mMlae@
139 2616 Dofsi,!mMlaf@
139 2617 Dofsi+!mMlaf@
139 2632 Eofri,!mMuafA
139 2639 Dofri,!mMuafA
139 2641 Dogsi,!mMuafA
139 2641 Dogsi,!mMuafA
139 2646 Dofti,!mMubfA
139 2649 Cofth,!mNlafA
139 2652 Dofsi,!mMuafA
139 2652 Dofsi,!mMuafA
139 2652 Dofsi,!mMuafA
139 2655 Dofsi,"mMuafA
139 2655 Dofsi,"mMuafA
139 2665 Dofsh,!mMuafA
139 2665 Dofsh,!mMuafA
139 2667 Dofti,!mNvalA
139 2667 Dofti,!mMuafA
139 2674 Dofti,!mMvafA

140 2588 Cofti,!mNubfA
140 2602 Dogsi,!mMlae@
140 2616 Dofsi,!mMlaf@
140 2617 Dofsi+!mMlaf@
140 2632 

169 2600 Dmfri,!mMuaeA
169 2602 Dogsi,!mMlae@
169 2603 Dogsi+!mMlae@
169 2605 Dogsi,!mMlaf@
169 2605 Dogsi,!mMlaf@
169 2612 Bogri,!mMlaf@
169 2616 Dofsi,!mMlaf@
169 2616 Dofsi,!mMlaf@
169 2616 Dofsi,!mMlaf@
169 2617 Dofsi+!mMlaf@
169 2617 Dofsi+!mMlaf@
169 2620 Dnfri,!mMuafA
169 2625 Bogsi,!mMlaf@

170 2530 Dnfsi,!mNlaf@
170 2558 Bogsi,!mNlaf@
170 2558 Bogsi,!mNlaf@
170 2573 Bogsi,!mNubfA
170 2587 Cogsi,!mMuaf@
170 2588 Cofti,!mNubfA
170 2588 Cofti,!mNubfA
170 2595 Dofsi,!mMlbf@
170 2600 Dmfri,!mMuaeA
170 2602 Dogsi,!mMlae@
170 2603 Dogsi+!mMlae@
170 2605 Dogsi,!mMlaf@
170 2605 Dogsi,!mMlaf@
170 2612 Bogri,!mMlaf@
170 2616 Dofsi,!mMlaf@
170 2616 Dofsi,!mMlaf@
170 2616 Dofsi,!mMlaf@
170 2617 Dofsi+!mMlaf@
170 2617 Dofsi+!mMlaf@
170 2620 Dnfri,!mMuafA

171 2530 Dnfsi,!mNlaf@
171 2554 Bogsi,!mNucfA
171 2558 Bogsi,!mNlaf@
171 2558 Bogsi,!mNlaf@
171 2573 Bogsi,!mNubfA
171 2587 Cogsi,!mMuaf@
171 2588 Cofti,!mNubfA
171 2588 Cofti,!mNubfA
171 2595 Dofsi,!mMlbf@
171 2600 Dmfri,!mMuaeA
171 2602 

242 2488 Dnhsj,!lNtaf@
242 2489 Dnhsi-!mNuag@
242 2492 Cnfsi+!mNubf@
242 2492 Cnfsi+!mNubf@
242 2495 Dogsj,!mNtaf@
242 2497 Dngsj,!lNtaf@
242 2499 Dmhsi-!mNlag@
242 2500 Dnfsi,!mNuae@
242 2500 Cogsi-!mNubf@
242 2506 Dnhsi,!mNu`f@

243 2446 Dnhsj,!mNtbf@
243 2447 Dogsi,!nNtcf@
243 2456 Cogsi,!nNtcf@
243 2461 Cogsi,!nNucf@
243 2464 Dnhsi,!nNuaf@
243 2467 Dnhsj,!mNtaf@
243 2475 Cmfsi+!mNubf@
243 2483 Dnfsi+!mNubf@
243 2484 Dnhsi-!mNuaf@
243 2488 Cogsj,!mNubf@
243 2488 Dnhsj,!lNtaf@
243 2489 Dnhsi-!mNuag@
243 2492 Cnfsi+!mNubf@
243 2492 Cnfsi+!mNubf@
243 2494 Bogsj,!mNtbf@
243 2495 Dogsj,!mNtaf@
243 2497 Dngsj,!lNtaf@
243 2499 Dmhsi-!mNlag@
243 2500 Dnfsi,!mNuae@
243 2500 Cogsi-!mNubf@

244 2446 Dnhsj,!mNtbf@
244 2447 Dogsi,!nNtcf@
244 2456 Cogsi,!nNtcf@
244 2461 Cogsi,!nNucf@
244 2464 Dnhsi,!nNuaf@
244 2467 Dnhsj,!mNtaf@
244 2475 Cmfsi+!mNubf@
244 2483 Dnfsi+!mNubf@
244 2484 Dnhsi-!mNuaf@
244 2488 Cogsj,!mNubf@
244 2488 Dnhsj,!lNtaf@
244 2489 Dnhsi-!mNuag@
244 2492 Cnfsi+!mNubf@
244 2492 

...

3254 2 Hfllo, world 
3254 2 Gello, wnrld!
3254 2 Hemmo, world!
3254 2 Hello, wnrld"
3254 2 Hfllo, world 
3254 2 Hemlo, wnrld!
3254 2 Gello, wnrld!
3254 2 Hemlo, wnrld!

1 Dofsi,!mMlRlN 4295
2 Dofsi,!mMlRlN 4295
3 Dofsi,!mMlRlN 4295
4 Dofsi,!mMlRlN 4295
5 Dofsi,!mMlRlN 4295
6 Dofsi,!mMlRlN 4295
7 Dofsi,!mMlRlN 4295
8 Dofsi,!mMlRlN 4295
9 Dofsi,!mMlRlN 4295
10 Dofsi,!mMlRlN 4295
11 Dofsi,!mMlRlN 4295
12 Dofsi,!mMlRlN 4295
13 Dofsi,!mMlRlN 4295
14 Dofsi,!mMlRlN 4295
15 Dofsi,!mMlRlN 4295
16 Dofsi,!mMlRlN 4295
17 Dofsi,!mMlRlN 4295
18 Dofsi,!mMlRlN 4295
19 Dofsi,!mMlRlN 4295
20 Dofsi,!mMlRlN 4295
21 Dofsi,!mMlRlN 4295
22 Dofsi,!mMlRlN 4295
23 Dofsi,!mMlRlN 4295
24 Dofsi,!mMlRlN 4295
25 Dofsi,!mMlRlN 4295
26 Dofsi,!mNlRlN 4228
27 Dofsi,!mNlRlN 4228
28 Dofsi,!mNlRlN 4228
29 Dofsi,!mNlRlN 4228
30 Dofsi,!mNlRlN 4228
31 Dofsi,!mNlRlN 4228
32 Dofsi,!mNlRlN 4228
33 Dofsi,!mNlRlN 4228
34 Dofsi,!mNlRlN 4228
35 Dofsi,!mNlRlN 4228
36 Dofsi,!mNlRlN 4228
37 Dofsi,!lMlRlC 3447
38 Dofsi,!lMlRlC 3447
39 D

830 Dhhqk+"oUsgf; 1533
831 Dhhqk+"oUsgf; 1533
832 Dhhqk+"oUsgf; 1533
833 Dhhqk+"oUsgf; 1533
834 Dhhqk+"oUsgf; 1533
835 Dhfql+!oUtff: 1506
836 Dhfql+!oUtff: 1506
837 Dhfql+!oUtff: 1506
838 Dhfql+!oUtff: 1506
839 Dhfql+!oUtff: 1506
840 Dhfql+!oUtff: 1506
841 Dhfql+!oUtff: 1506
842 Dhfql+!oUtff: 1506
843 Dhfql+!oUtff: 1506
844 Dhfql+!oUtff: 1506
845 Dhfql+!oUtff: 1506
846 Dhfql+!oUtff: 1506
847 Dhfql+!oUtff: 1506
848 Dhfql+!oUtff: 1506
849 Dhfql+!oUtff: 1506
850 Dhfql+!oUtff: 1506
851 Dhfql+!oUtff: 1506
852 Dhfql+!oUtff: 1506
853 Dhfql+!oUtff: 1506
854 Dhfql+!oUtff: 1506
855 Dhfql+!oUtff: 1506
856 Dhfql+!oUtff: 1506
857 Dhfql+!oUtff: 1506
858 Dhfql+!oUtff: 1506
859 Dhfql+!oUtff: 1506
860 Dhfql+!oUtff: 1506
861 Dhfql+!oUtff: 1506
862 Dhfql+!oUtff: 1506
863 Dhfql+!oUtff: 1506
864 Dhfql+!oUtff: 1506
865 Dhfql+!oUtff: 1506
866 Dhfql+!oUtff: 1506
867 Dhfql+!oUtff: 1506
868 Dhfql+!oUtff: 1506
869 Dhfql+!oUtff: 1506
870 Dhfql+!oUtff: 1506
871 Dhfql+!oUtff: 1506
872 Dhfql+!oUtff: 1506
873 Dhfql+!

1241 Dghol,"pZthf6 1013
1242 Dghol,"pZthf6 1013
1243 Dghol,"pZthf6 1013
1244 Dghol,"pZthf6 1013
1245 Dghol,"pZthf6 1013
1246 Dghol,"pZthf6 1013
1247 Dghol,"pZthf6 1013
1248 Dghol,"pZthf6 1013
1249 Dghol,"pZthf6 1013
1250 Dghol,"pZthf6 1013
1251 Dghol,"pZthf6 1013
1252 Dghol,"pZthf6 1013
1253 Dghol,"pZthf6 1013
1254 Dghol,"pZthf6 1013
1255 Dghol,"pZthf6 1013
1256 Dghol,"pZthf6 1013
1257 Dghol,"pZthf6 1013
1258 Dghol,"pZthf6 1013
1259 Dghol,"pZthf6 1013
1260 Dghol,"pZthf6 1013
1261 Dghol,"pZthf6 1013
1262 Dghol,"pZthf6 1013
1263 Dghol,"pZthf6 1013
1264 Dghol,"pZthf6 1013
1265 Dghol,"pZthf6 1013
1266 Dghol,"pZthf6 1013
1267 Dghol,"pZthf6 1013
1268 Dghol,"pZthf6 1013
1269 Dghol,"pZthf6 1013
1270 Dghol,"pZthf6 1013
1271 Dghol,"pZthf6 1013
1272 Dghol,"pZthf6 1013
1273 Dghol,"pZthf6 1013
1274 Dghol,"pZthf6 1013
1275 Dghol,"pZthf6 1013
1276 Dghol,"pZthf6 1013
1277 Dghol,"pZthf6 1013
1278 Dghol,"pZthf6 1013
1279 Dghol,"pZthf6 1013
1280 Dghol,"pZthf6 1013
1281 Dghol,"pZthf6 1013
1282 Dghol,"pZth

1740 Dglnp-#s]sie2 675
1741 Dglnp-#s]sie2 675
1742 Dglnp-#s]sie2 675
1743 Dglnp-#s]sie2 675
1744 Dglnp-#s]sie2 675
1745 Dglnp-#s]sie2 675
1746 Dglnp-#s]sie2 675
1747 Dglnp-#s]sie2 675
1748 Dglnp-#s]sie2 675
1749 Delnn-"s]she2 673
1750 Delnn-"s]she2 673
1751 Dglmn-"s]she1 641
1752 Dglmn-"s]she1 641
1753 Dglmn-"s]she1 641
1754 Dglmn-"s]she1 641
1755 Dglmn-"s]she1 641
1756 Dglmn-"s]she1 641
1757 Dglmn-"s]she1 641
1758 Dflmn-"s]she1 638
1759 Dflmn-"s]she1 638
1760 Dflmn-"s]she1 638
1761 Dflmn-"s]she1 638
1762 Dflmn-"s]she1 638
1763 Dflmn-"s]she1 638
1764 Dflmn-"s]she1 638
1765 Dflmn-"s]she1 638
1766 Delno-"s^she2 637
1767 Delnn.#s^she1 613
1768 Delnn.#s^she1 613
1769 Delnn.#s^she1 613
1770 Delnn.#s^she1 613
1771 Delnn.#s^she1 613
1772 Delnn.#s^she1 613
1773 Delnn.#s^she1 613
1774 Delnn.#s^she1 613
1775 Delnn.#s^she1 613
1776 Delnn.#s^she1 613
1777 Delnn.#s^she1 613
1778 Delnn.#s^she1 613
1779 Delnn.#s^she1 613
1780 Delnn.#s^she1 613
1781 Delnn.#s^she1 613
1782 Delnn.#s^she1 613
1783 Delnn.

2574 Eenlo-"uitkd% 79
2575 Eenlo-"uitkd% 79
2576 Eenlo-"uitkd% 79
2577 Eenlo-"uitkd% 79
2578 Eenlo-"uitkd% 79
2579 Eenlo-"uitkd% 79
2580 Eenlo-"uitkd% 79
2581 Eenlo-"uitkd% 79
2582 Eenlo-"uitkd% 79
2583 Eenlo-"uitkd% 79
2584 Eenlo-"uitkd% 79
2585 Eenlo-"uitkd% 79
2586 Eenlo-"uitkd% 79
2587 Eenlo-"uitkd% 79
2588 Eenlo-"uitkd% 79
2589 Eenlo-"uitkd% 79
2590 Eenlo-"uitkd% 79
2591 Eenlo-"uitkd% 79
2592 Eenlo-"uitkd% 79
2593 Eenlo-"uitkd% 79
2594 Eenlo-"uitkd% 79
2595 Eenlo-"uitkd% 79
2596 Eenlo-"uitkd% 79
2597 Eenlo-"uitkd% 79
2598 Eenlo-"uitkd% 79
2599 Eenlo-"uitkd% 79
2600 Eenlo-"uitkd% 79
2601 Eenlo-"uitkd% 79
2602 Eenlo-"uitkd% 79
2603 Eenlo-"uitkd% 79
2604 Eenlo-"uitkd% 79
2605 Eenlo-"uitkd% 79
2606 Eenlo-"uitkd% 79
2607 Eenlo-"uitkd% 79
2608 Eenlo-"uitkd% 79
2609 Eenlo-"uitkd% 79
2610 Eenlo-"uitkd% 79
2611 Eenlo-"uitkd% 79
2612 Eenlo-"uitkd% 79
2613 Eenlo-"uitkd% 79
2614 Eenlo-"uitkd% 79
2615 Eenlo-"uitkd% 79
2616 Eenlo-"uitkd% 79
2617 Eenln-"uirkd% 76
2618 Eenln-"uirkd% 76
2619 Eenln

# Activity: What happens when we extremify the genetic mutations?

# Activity: What happens when we mutate more than one character?